<span style="color:#FF7A00; font-size:30px; font-weight:bold;">
BDP Term Project (1조)
</span>

<span style="font-size:20px; font-weight:bold;">
encar.com에서 국산 중고차 매물별 보험이력 불러오기
</span>

### Driver Load

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import csv
import time

print("작업 완료")

작업 완료


In [2]:
# Chrome 브라우저의 옵션을 설정
chrome_options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

# Chrome 드라이버 생성시 옵션을 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

### load csv
#### 사용방법
- `file_path`를 수정
- `ids_file_path` = `id, price`가 포함되어있는 `csv`파일의 경로 / 기본값: `"../export/vehicles_id_price.csv"`
- `output_file_path` = 결과물을 저장할 파일의 경로
#### 코드설명
- 프로젝트 내 `export` 폴더 내에 있는 `vehicles_id_price.csv` 에서 전체 매물의 `id`를 가져옵니다.
- 이미 크롤링이 완료된 `id`를 제외해서 `ids` 변수에 저장합니다.

In [14]:
ids_file_path = "../export/vehicles_id_price.csv"
output_file_path = "../export/insurance_all.csv"

all_ids = set()
try:
    with open(ids_file_path, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            all_ids.add(row["id"])
except:
    print(f"{ids_file_path} 파일이 존재하지 않습니다.")


existing_ids = set()
try:
    with open(output_file_path, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            existing_ids.add(row["ID"])
except FileNotFoundError:
    print(f"{output_file_path} 파일이 존재하지 않습니다.")

if len(all_ids) == 0 or len(existing_ids) == 0:
    print("파일 경로를 다시 입력해주세요.")
else:
    ids = all_ids - existing_ids
    print(ids)

{'38528247', '38604094', '38503790', '36376572', '38166516', '37806683', '38446206', '38155932', '38043348', '38604422', '38617765', '38580329', '37912194', '38531169', '38257958', '38457602', '38463362', '38328286', '38451113', '38268606', '38069452', '38268733', '38587340', '38474560', '38589471', '38362756', '38607701', '38065128', '38555892', '38420016', '38082868', '38544172', '38138991', '38553084', '38412519', '38330416', '38634708', '37522092', '36200229', '38522061', '37197086', '38559028', '38301062', '38273444', '38481359', '38451089', '37628806', '38328406', '38577695', '38398474', '38540178', '38506316', '38498621', '38442835', '38616194', '38371077', '38359255', '38356345', '38448515', '38281481', '38301785', '38604016', '38625059', '38514297', '37283641', '36360908', '37833551', '38075574', '38426088', '37755461', '38088513', '38583038', '38386577', '38536096', '38269199', '38553466', '38604641', '38600439', '38310482', '38160215', '38295702', '38341259', '38574640', '38

### crawling

In [4]:
def main():
    base_url = "https://fem.encar.com/cars/report/accident/{id}" # 보험이력 조회 링크
    # base_url = "https://fem.encar.com/cars/report/inspect/{id}" # 성능점검 조회 링크
    vehicles = fetch_vehicle_ids(driver, base_url, max_iter)
        
    output_file = "vehicles_id_price.csv"
    try:
        with open(output_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(["id", "price"])  # 헤더 작성
            for vehicle_id, price in vehicles.items():
                writer.writerow([vehicle_id, price])
        print(f"매물 데이터 {len(vehicles)}개가 {output_file} 파일에 저장되었습니다.")
    except Exception as e:
        print("CSV 저장 중 오류 발생:", e)

def fetch_vehicle_ids(driver, base_url, max_iter = 1):
    vehicles = {}
    
    for page in range(1, max_iter + 1):
        driver.get(base_url.format(page = page))
    
        # 요소 팝업까지 대기 및 무결성 검사 
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-impression]")))
            elements = driver.find_elements(By.XPATH, "//div[@data-impression]")
        except Exception:
            print(f"페이지 {page} 로드 실패. 크롤링을 중단합니다.")
            break

        # 페이지에 데이터 추출 후 dict에 보관
        for e in elements:
            data_impression = e.get_attribute("data-impression")
            if data_impression:
                # data-impression 값을 분리하여 id와 price 추출
                parts = data_impression.split("|")
                if len(parts) >= 2:
                    vehicle_id = parts[0]  # ID
                    price = parts[1]       # 가격
                    # 중복되지 않게 저장
                    vehicles[vehicle_id] = price
    return vehicles

if __name__ == "__main__":
    main()

페이지 595 로드 실패. 크롤링을 중단합니다.
매물 데이터 118419개가 vehicles_id_price.csv 파일에 저장되었습니다.


In [5]:
try:
    driver.close()
except:
    print("driver 연결 오류")